<a href="https://colab.research.google.com/github/Ricardo0621/04-05-2017-SO/blob/master/examples/COLAB/COLAB_maDLC_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut 2.2+ Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628180434489-T0RIWEJJU0FJVOT6FNVD/maDLC.png?format=800w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- analyze novel videos
- assemble animals and tracklets
- create quality check plots!

###This notebook assumes you already have a DLC project folder with labeled data and you uploaded it to your own Google Drive.

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub. We also recommend checking out our preprint, which covers the science of maDLC

**Lauer et al 2021:** https://www.biorxiv.org/content/10.1101/2021.04.30.442096v1




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

     |████████████████████████████████| 544 kB 13.8 MB/s 
     |████████████████████████████████| 29.2 MB 1.2 MB/s 
     |████████████████████████████████| 109 kB 63.2 MB/s 
     |████████████████████████████████| 9.8 MB 52.6 MB/s 
     |████████████████████████████████| 352 kB 66.4 MB/s 
     |████████████████████████████████| 4.3 MB 55.8 MB/s 
     |████████████████████████████████| 948 kB 64.0 MB/s 
     |████████████████████████████████| 296 kB 78.2 MB/s 
     |████████████████████████████████| 177 kB 75.8 MB/s 
     |████████████████████████████████| 546 kB 66.5 MB/s 
     |████████████████████████████████| 462 kB 67.9 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=dd5ff1800a9a36d6cc211775a6267ea6068c63ed1a850b84e406d9f1cf0fe30a
  Stored in directory: /root/.cache/pip/wheels/ce/e0/ee/a2b3c5caab3418c1ccd8c4de573d4cbe13315d7e8b0a55fbc2
Successfully built filterpy
  Attempting uninstall: scikit-image
    Found existing installation: sc

In [2]:
#a few colab specific things needed:
!pip install --upgrade scikit-image
!pip3 install pickle5

     |████████████████████████████████| 13.5 MB 7.3 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.1
    Uninstalling scikit-image-0.18.1:
      Successfully uninstalled scikit-image-0.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deeplabcut 2.2.0.6 requires scikit-image<=0.18.1,>=0.17, but you have scikit-image 0.19.2 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.
     |████████████████████████████████| 256 kB 14.9 MB/s 


After the package is installed, please click "restart runtime" if it appears for DLC changes to take effect in your COLAB environment. You will see this button at the output of the cells above ^.

In [3]:
import deeplabcut
import pickle5 as pickle

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


/usr/local/lib/python3.7/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


## Link your Google Drive (with your labeled data):

- This code assumes you locally installed DeepLabCut, created a project, extracted and labeled frames. Be sure to "check Labels" to confirm you are happy with your data. As, these frames are the only thing that is used to train your network. 💪 You can find all the docs to do this here: https://deeplabcut.github.io/DeepLabCut

- Next, place your DLC project folder into you Google Drive- i.e., copy the folder named "Project-YourName-TheDate" into Google Drive.

- Then, click run on the cell below to link this notebook to your Google Drive:

In [4]:
#Now, let's link to your Google Drive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Next, edit the few items below, and click run:


In [5]:
# PLEASE EDIT THIS:
ProjectFolderName = 'Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23'
VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# No need to edit this, we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/drive/My Drive/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path

#No need to edit this, as you set it when you passed the ProjectFolderName (above): 
path_config_file = '/content/drive/My Drive/'+ProjectFolderName+'/config.yaml'
path_config_file
#This creates a path variable that links to your google drive project

'/content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/config.yaml'

## Create a multi-animal training dataset:

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#create-training-dataset

- please check the text below, edit if needed, and then click run (this can take some time):

In [7]:
#OPTIONAL LEARNING: did you know you can check what each function does by running with a ?
deeplabcut.create_multianimaltraining_dataset?

In [6]:
# ATTENTION:
#which shuffle do you want to create and train?
shuffle = 1 #edit if needed; 1 is the default.

#if you labeled on Windows, please set the windows2linux=True:
deeplabcut.create_multianimaltraining_dataset(path_config_file, Shuffles=[shuffle], net_type="dlcrnet_ms5",windows2linux=False)

Utilizing the following graph: [[0, 1], [0, 2], [1, 2]]
Creating training data for: Shuffle: 1 TrainFraction:  0.95


100%|██████████| 38/38 [00:18<00:00,  2.00it/s]


The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training:
This function trains the network for a specific shuffle of the training dataset. 
 - more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#train-the-network

In [7]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#Typically, you want to train to 50,000 - 200K iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=100,saveiters=10000, maxiters=50000, allow_growth=True)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 50K iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['head', 'left_wing', 'right_wing'],
 'alpha_r': 0.02,
 'apply_prob': 0.5,
 'batch_size': 8,
 'clahe': True,
 'claheratio': 0.1,
 'crop_pad': 0,
 'crop_sampling': 'hybrid',
 'crop_size': [400, 400],
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Fly_fight_analysisFeb23/Fly_fight_analysis_Ricardo '
            'Diaz-Rincon95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 500,
 'edge': False,
 'emboss': {'alpha': [0.0, 1.0], 'embossratio': 0.1, 'strength': [0.5, 1.5]},
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'histeq': True,
 'histeqratio': 0.1,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_

Selecting multi-animal trainer
Activating limb prediction...
Batch Size is 8
Getting specs multi-animal-imgaug 3 3


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Loading ImageNet-pretrained resnet_50
Max_iters overwritten as 50000
Display_iters overwritten as 100
Save_iters overwritten as 10000
Training parameters:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/dlc-models/iteration-0/Fly_fight_analysisFeb23-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'adam', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 8, 'dataset_type': 'multi-animal-imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_only_present_joints': False, 'partaffinityfield_predic

iteration: 100 loss: 0.0584 scmap loss: 0.0461 locref loss: 0.0028 limb loss: 0.0094 lr: 0.0001
iteration: 200 loss: 0.0186 scmap loss: 0.0164 locref loss: 0.0005 limb loss: 0.0016 lr: 0.0001
iteration: 300 loss: 0.0129 scmap loss: 0.0113 locref loss: 0.0004 limb loss: 0.0012 lr: 0.0001
iteration: 400 loss: 0.0111 scmap loss: 0.0098 locref loss: 0.0003 limb loss: 0.0010 lr: 0.0001
iteration: 500 loss: 0.0098 scmap loss: 0.0087 locref loss: 0.0003 limb loss: 0.0008 lr: 0.0001
iteration: 600 loss: 0.0086 scmap loss: 0.0076 locref loss: 0.0003 limb loss: 0.0007 lr: 0.0001
iteration: 700 loss: 0.0079 scmap loss: 0.0070 locref loss: 0.0002 limb loss: 0.0007 lr: 0.0001
iteration: 800 loss: 0.0076 scmap loss: 0.0068 locref loss: 0.0002 limb loss: 0.0006 lr: 0.0001
iteration: 900 loss: 0.0079 scmap loss: 0.0071 locref loss: 0.0002 limb loss: 0.0006 lr: 0.0001
iteration: 1000 loss: 0.0071 scmap loss: 0.0063 locref loss: 0.0002 limb loss: 0.0005 lr: 0.0001
iteration: 1100 loss: 0.0067 scmap loss

The network is now trained and ready to evaluate. Use the function 'evaluate_network' to evaluate the network.


Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/core/train.py", line 83, in load_and_enqueue
    sess.run(enqueue_op, feed_dict=food)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py", line 968, in run
    run_metadata_ptr)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py", line 1115, in _run
    raise RuntimeError('Attempted to use a closed Session.')
RuntimeError: Attempted to use a closed Session.



**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating: 

 - First, we evaluate the pose estimation performance.

- This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#evaluate-the-trained-network

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [8]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['head', 'left_wing', 'right_wing'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Fly_fight_analysisFeb23/Fly_fight_analysis_Ricardo '
            'Diaz-Rincon95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_smooth': False,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'multi_stage': True,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_idchannel': 0,
 'num_joints': 3,
 'num_limbs': 3,
 'optimizer': 'sgd',
 'pairwi

Running  DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000  with # of trainingiterations: 50000
Activating extracting of PAFs
Network Evaluation underway...


40it [00:19,  2.06it/s]


Results for 50000 training iterations, training fraction of 95, and shuffle 1:
Train error: 2.55 pixels. Test error: 3.54 pixels.
With pcutoff of 0.6:
Train error: 2.55 pixels. Test error: 3.54 pixels.
##########################################
Average Euclidean distance to GT per individual (in pixels)
individuals
W1_BF    2.337299
W1_WF    2.659954
W2_BF    2.747308
W2_WF    2.594365
W3_BF    2.821412
W3_WF    2.563001
Average Euclidean distance to GT per bodypart (in pixels)
bodyparts
head          2.762688
left_wing     2.563501
right_wing    2.458569
Done and results stored for snapshot:  snapshot-50000
Selecting best skeleton...
Graph 1|2


100%|██████████| 40/40 [00:00<00:00, 1532.53it/s]

...


100%|██████████| 40/40 [00:00<00:00, 1709.08it/s]
Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['head', 'left_wing', 'right_wing'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Fly_fight_analysisFeb23/Fly_fight_analysis_Ricardo '
            'Diaz-Rincon95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_smooth': False,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'multi_stage': True,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_idchannel': 0,
 'num_joints'

Activating extracting of PAFs
Analyzing data...


1it [00:00,  2.64it/s]


Saving plots...


100%|██████████| 1/1 [00:01<00:00,  1.89s/it]


IF these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train! 

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in a pickle file in the same directory where the video resides. 

In [10]:
print("Start Analyzing my video(s)!")
#EDIT OPTION: which video(s) do you want to analyze? You can pass a path or a folder:
# currently, if you run "as is" it assumes you have a video in the DLC project video folder!

deeplabcut.analyze_videos(path_config_file,videofile_path, shuffle=shuffle, videotype=VideoType)

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['head', 'left_wing', 'right_wing'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Fly_fight_analysisFeb23/Fly_fight_analysis_Ricardo '
            'Diaz-Rincon95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_smooth': False,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'multi_stage': True,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_idchannel': 0,
 'num_joints': 3,
 'num_limbs': 3,
 'optimizer': 'sgd',
 'paf_be

Start Analyzing my video(s)!
Using snapshot-50000 for model /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/dlc-models/iteration-0/Fly_fight_analysisFeb23-trainset95shuffle1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Activating extracting of PAFs


Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['head', 'left_wing', 'right_wing'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Fly_fight_analysisFeb23/Fly_fight_analysis_Ricardo '
            'Diaz-Rincon95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_smooth': False,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'multi_stage': True,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_idchannel': 0,
 'num_joints': 3,
 'num_limbs': 3,
 'optimizer': 'sgd',
 'paf_be

Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004.mp4
Video already analyzed! /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000.h5
Using snapshot-50000 for model /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/dlc-models/iteration-0/Fly_fight_analysisFeb23-trainset95shuffle1
Processing...  /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004.mp4
Tracklets already computed /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000_el.pickle
Set overwrite = True to overwrite.
The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Process

100%|██████████| 1/1 [00:00<00:00, 6808.94it/s]
/usr/local/lib/python3.7/dist-packages/deeplabcut/refine_training_dataset/stitch.py:671: UserWarning: No optimal solution found. Employing black magic...
  warnings.warn("No optimal solution found. Employing black magic...")


ValueError: ignored

Optional: Now you have the option to check the raw dections before animals are assembled. To do so, pass a video path:

In [12]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
## don't look good, don't proceed with tracking - add more data to your training set and re-train!

#EDIT: let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = '/content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004.mp4'

#don't edit:
deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], shuffle=shuffle)

Detections already plotted,  /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000_full.mp4


If the resutling video (ends in full.mp4) is not good, we highly recommend adding more data and training again. See here: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#decision-break-point

In [ ]:
pickle_file = ''
deeplabcut.find_outliers_in_raw_data(path_config_file, pickle_file, Specific_videofile)

# Next, we will assemble animals using our data-driven optimal graph method:

- Here, we will find the optimal graph, which matches the "data-driven" method from our paper (Figure adapted from Lauer et al. 2021):

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1626266017809-XO6NX84QB4FBAZGOTCEY/fig3.jpg?format=400w)


- note, you can set the number of animals you expect to see, so check, edit, then click run:

In [14]:
#Check and edit:
numAnimals = 6 #how many animals do you expect to find?
tracktype= 'box' #box, skeleton, ellipse:
#-- ellipse is recommended, unless you have a single-point ma project, then use BOX!

#Optional: 
#imagine you tracked a point that is not useful for assembly, 
#like a tail tip that is far from the body, consider dropping it for this step (it's still used later)!
#To drop it, uncomment the next line TWO lines and add your parts(s):

#bodypart= 'Tail_end'
#deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, overwrite=True, ignore_bodyparts=[bodypart])

#OR don't drop, just click RUN:
deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, 
                                        shuffle=shuffle, overwrite=True)

deeplabcut.stitch_tracklets(path_config_file, videofile_path, shuffle=shuffle, track_method=tracktype, n_tracks=numAnimals)

Config:
{'all_joints': [[0], [1], [2]],
 'all_joints_names': ['head', 'left_wing', 'right_wing'],
 'batch_size': 1,
 'crop_pad': 0,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_Fly_fight_analysisFeb23/Fly_fight_analysis_Ricardo '
            'Diaz-Rincon95shuffle1.pickle',
 'dataset_type': 'multi-animal-imgaug',
 'deterministic': False,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 1.0,
 'locref_smooth': False,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'mean_pixel': [123.68, 116.779, 103.939],
 'minconfidence': 0.01,
 'mirror': False,
 'multi_stage': True,
 'net_type': 'resnet_50',
 'nmsradius': 5.0,
 'num_idchannel': 0,
 'num_joints': 3,
 'num_limbs': 3,
 'optimizer': 'sgd',
 'paf_be

Using snapshot-50000 for model /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/dlc-models/iteration-0/Fly_fight_analysisFeb23-trainset95shuffle1
Analyzing all the videos in the directory...
Processing...  /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004.mp4
Analyzing /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000.h5


100%|██████████| 6780/6780 [00:03<00:00, 2105.53it/s]
6780it [00:00, 51927.33it/s]

The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Analyzing all the videos in the directory...
No video(s) found. Please check your path!


Now let's filter the data to remove any small jitter:

In [15]:
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 shuffle=shuffle,
                                 videotype=VideoType, 
                                 track_method = tracktype)

Analyzing all the videos in the directory...
Filtering with median model /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004.mp4
No unfiltered data file found in /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos for video 00004 and scorer DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000 and box tracker.


## Create plots of your trajectories:

In [16]:
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, track_method=tracktype)

Analyzing all the videos in the directory...
Loading  /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004.mp4 and data.
No unfiltered data file found in /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos for video 00004 and scorer DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000 and box tracker.
No detection data found in /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos for video 00004, scorer DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000, and tracker box
Make sure /content/drive/My Drive/Fly_fight_analysis-Ricardo Diaz-Rincon-2022-02-23/videos/00004.mp4 was previously analyzed, and that detections were successively converted to tracklets using "deeplabcut.convert_detections2tracklets()" and "deeplabcut.stitch_tracklets()".
Plots could not be created! Videos were not evaluated with the current scorer DLC_dlcrnetms5_Fly_fight_analysisFeb23shuffle1_50000.


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                color_by="individual",
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True)